# Homework 7

## Due Thursday, November 29th 2018 at 11:59 PM.

### Be sure to push the final version of your notebook to your GitHub repo.  Follow the instructions on the course website.

### Topics
####  [Part 1](#part_1):  Database schema [15 points]
* [Problem 1](#p1.1). Schema [15 points]

####  [Part 2](#part_2):  Insert records [35 points]
* [Problem 2](#p2.1). Baseline model [15 points]
* [Problem 3](#p2.2). Reduced model [10 points]
* [Problem 4](#p2.3). L1 penalty model [10 points]

####  [Part 3](#part_3):  Queries [20 pts]
* [Problem 5](#p3.1). Best model coefficients [10 points]
* [Problem 6](#p3.2). Best model score [10 points]

---

<a id='part_1'></a>
# Part 1:  Database schema

<a id='p1.1'></a>
## Problem 1 (15 points): 

In this problem you will set up a SQL database using the `sqllite` package in Python. The purpose of the database will be to store parameters and model results related to a simple *Logistic Regression* problem. Rather than keeping the results in `Numpy` arrays as we usually do, the idea here is to make use of a `SQL` database to materialize the results so that it can easily be accessed from disk at a later stage.

The design of the database should be flexible enough so that the results from different model iterations can be stored in the database. It should also be able to deal with a different set of features by model iteration.

A list of the tables to include in the database and the relevant fields in each table is shown below (tables are in bold):

**model_params**: 
* id 
* desc 
* param_name
* value

**model_coeffs**
* id 
* desc 
* feature_name
* value

**model_results**
* id 
* desc 
* train_score
* test_score

Create a `SQL` database called `regression.sqlite` containing the three tables shown above.

In [194]:
import sqlite3

db = sqlite3.connect('regression.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS model_params")
cursor.execute("DROP TABLE IF EXISTS model_coeffs")
cursor.execute("DROP TABLE IF EXISTS model_results")

# Create model_params Table
cursor.execute('''CREATE TABLE model_params (
               id INTEGER NOT NULL, 
               desc TEXT, 
               param_name TEXT, 
               value REAL)''')

# Create model_coeffs Table
cursor.execute('''CREATE TABLE model_coeffs (
               id INTEGER NOT NULL, 
               desc TEXT, 
               feature_name TEXT, 
               value REAL)''')

# Create model_results Table
cursor.execute('''CREATE TABLE model_results (
               id INTEGER NOT NULL, 
               desc TEXT, 
               train_score REAL, 
               test_score REAL)''')

db.commit()

In [195]:
# Helper function to visualize tables
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

<a id='part_2'></a>
# Part 2: Insert records

In this section you will populate the database you created in the previous question with some records for a number of different model iterations / scenarios.

<a id='p2.1'></a>
## Problem 2 (15 points): 
Create a baseline Logistic Regression model using the provided code (below).  Insert the relevant arrays into the corresponding tables in the database.

**model_params**
Values from the [`get_params`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.get_params) method.

**model_coeffs**
Coefficient and intercept values of the fitted model (see `coef_` and `intercept_` attributes in the documentation).

**model_results**
Train and validation accuracy obtained from the [`score`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score) method.


#### Remarks
* Reference scikit-learn documentation to get more detail on the methods / attributes list above:  
[https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

* Note that the *id* and *desc* are just identifier fields used to identify the results from a specific model iteration or scenario. For example for the baseline model you could set *id = 1* and *desc = "Baseline model"*.


#### Suggestions
You may want to create a function to save data to the database.  You will be able to re-use this function in subsequent sections.

In [196]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer

%matplotlib inline

In [197]:
# Load data
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)

In [198]:
# Fit model
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [199]:
def save_data(id_val=1, desc_val="Baseline model", X=X, penalty_val="l2"):
    
    # Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)
    
    # Fit model
    model = LogisticRegression(penalty = penalty_val)
    model.fit(X_train, y_train)
    
    # Fill in model_params table
    params = model.get_params()
    for key, value in params.items():
        row = (id_val, desc_val, key, value)
        sql = ''' INSERT INTO model_params(id, desc, param_name, value)
              VALUES(?,?,?,?)'''
        cursor.execute(sql, row)

    # Fill in model_coeffs table
    coef_dict = {}
    for coef, feat in zip(model.coef_[0,:], list(X)):
        coef_dict[feat] = coef
    for key, value in coef_dict.items():
        row = (id_val, desc_val, key, value)
        sql = ''' INSERT INTO model_coeffs(id, desc, feature_name, value)
              VALUES(?,?,?,?)'''
        cursor.execute(sql, row)
    
    # Fill in intercept value in model_coeffs
    intercept = model.intercept_[0]
    row = (id_val, desc_val, "Intercept", intercept)
    sql = ''' INSERT INTO model_coeffs(id, desc, feature_name, value)
          VALUES(?,?,?,?)'''
    cursor.execute(sql, row)
    
    # Fill in model_results
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    row = (id_val, desc_val, train_score, test_score)
    sql = ''' INSERT INTO model_results(id, desc, train_score, test_score)
              VALUES(?, ?,?,?)'''
    cursor.execute(sql, row)
    db.commit()

In [200]:
save_data(1, "Baseline model")

<a id='p2.2'></a>
## Problem 3 (10 points): 
Create a second model using only the features included in the list below (in `feature_cols`).  Insert the relevant arrays into the corresponding tables in the database.

Remember to update the `id` and `desc` values for the second iteration.

#### Suggestions
* Name this second model `"Reduced model"`.

In [201]:
feature_cols = ['mean radius',
                'texture error',
                'worst radius',
                'worst compactness',
                'worst concavity']

X_reduced = X[feature_cols]

In [202]:
save_data(id_val=2, desc_val="Reduced model", X=X_reduced)

<a id='p2.3'></a>
## Problem 4 (10 points): 
Create one last model using an **l1-penalty** ($L_{1}$) term and **all** the features. Insert the relevant arrays into the corresponding tables in the database.

**Hint:** Refer to the `penalty` parameter of the `LogisticRegression` class.

#### Suggestions
Call this model `"L1 penalty model"`.

In [203]:
save_data(id_val=3, desc_val="L1 penalty model", X=X, penalty_val="l1")

In [204]:
cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_params)")]
query = '''SELECT * FROM model_params'''
display(viz_tables(cols, query))

,id,desc,param_name,value
0,1,Baseline model,C,1
1,1,Baseline model,class_weight,None
2,1,Baseline model,dual,0
3,1,Baseline model,fit_intercept,1
4,1,Baseline model,intercept_scaling,1
5,1,Baseline model,max_iter,100
6,1,Baseline model,multi_class,ovr
7,1,Baseline model,n_jobs,1
8,1,Baseline model,penalty,l2
9,1,Baseline model,random_state,None


In [205]:
cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_coeffs)")]
query = '''SELECT * FROM model_coeffs'''
display(viz_tables(cols, query))

,id,desc,feature_name,value
0,1,Baseline model,mean radius,2.143352
1,1,Baseline model,mean texture,0.073687
2,1,Baseline model,mean perimeter,-0.148922
3,1,Baseline model,mean area,0.015650
4,1,Baseline model,mean smoothness,-0.104633
5,1,Baseline model,mean compactness,-0.407477
6,1,Baseline model,mean concavity,-0.594942
7,1,Baseline model,mean concave points,-0.263499
8,1,Baseline model,mean symmetry,-0.155283
9,1,Baseline model,mean fractal dimension,-0.028102


In [206]:
cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_results)")]
query = '''SELECT * FROM model_results'''
display(viz_tables(cols, query))

,id,desc,train_score,test_score
0,1,Baseline model,0.960440,0.938596
1,2,Reduced model,0.945055,0.885965
2,3,L1 penalty model,0.964835,0.947368


<a id='part_3'></a>

# Part 3:  Queries

<a id='p3.1'></a>
## Problem 5 (10 points): 
Query the database to identify the model with the highest validation score.
* Print the id of the best model and the corresponding validation score.
  ```bash
  Best model id: 
  Best validation score:
  ```
* Print the feature names and corresponding coefficients of that model.

In [207]:
func_best_model = '''SELECT id, MAX(test_score) as max_score FROM model_results'''
cursor.execute(func_best_model)
model_id, model_score = cursor.fetchone()
print(f"Best model id: {model_id}")
print(f"Best validation score: {model_score}")

Best model id: 3
Best validation score: 0.9473684210526315


In [208]:
display(viz_tables(['Best model id', 'Best validation score'], func_best_model))

,Best model id,Best validation score
0,3,0.947368


In [209]:
query = '''SELECT feature_name, value FROM model_coeffs WHERE id=3'''
cursor.execute(query)
coef_dict = {}
for name, value in cursor.fetchall():
    print(f"feature_name: {name}, coeff_value: {value}")
    coef_dict[name] = value

feature_name: mean radius, coeff_value: 6.144804745703106
feature_name: mean texture, coeff_value: 0.020949668866605646
feature_name: mean perimeter, coeff_value: -0.5109291130606262
feature_name: mean area, coeff_value: 0.00010866590651929544
feature_name: mean smoothness, coeff_value: 0.0
feature_name: mean compactness, coeff_value: 0.0
feature_name: mean concavity, coeff_value: 0.0
feature_name: mean concave points, coeff_value: 0.0
feature_name: mean symmetry, coeff_value: 0.0
feature_name: mean fractal dimension, coeff_value: 0.0
feature_name: radius error, coeff_value: 0.0
feature_name: texture error, coeff_value: 0.0
feature_name: perimeter error, coeff_value: 0.6845148859045012
feature_name: area error, coeff_value: -0.052584036331933745
feature_name: smoothness error, coeff_value: 0.0
feature_name: compactness error, coeff_value: 0.0
feature_name: concavity error, coeff_value: 0.0
feature_name: concave points error, coeff_value: 0.0
feature_name: symmetry error, coeff_value: 0

In [210]:
display(viz_tables(['feature_name', 'coeff_value'], query))

,feature_name,coeff_value
0,mean radius,6.144805
1,mean texture,0.020950
2,mean perimeter,-0.510929
3,mean area,0.000109
4,mean smoothness,0.000000
5,mean compactness,0.000000
6,mean concavity,0.000000
7,mean concave points,0.000000
8,mean symmetry,0.000000
9,mean fractal dimension,0.000000


<a id='p3.2'></a>
## Problem 6 (10 points): 

Use the coefficients extracted in the previous question to reproduce the validation score (accuracy) of the best performing model (as stored in the database).

**Hint:** You should be able to achieve this by overwriting the relevant variables in the Logistic regression object, i.e. there is no need write your own formula to generate individual predictions (you are welcome to do this if you want).

#### Remarks
The problem demos a simple scenario in which someone with access to your database can easily reproduce your results.

In [211]:
f = '''SELECT feature_name, value FROM model_coeffs WHERE id = ?'''
cursor.execute(f, (model_id,))
coeffs = cursor.fetchall()
coef_dict = {}
intercept_value = None
for feature_name, value in coeffs:
    if feature_name != "Intercept":
        coef_dict[feature_name] = value
    else:
        intercept_value = value

In [212]:
clf.coef_[0] = list(coef_dict.values())
clf.intercept_[0] = intercept_value

In [213]:
clf.score(X_test,y_test)

0.9473684210526315